In [1]:
import jax.numpy as jnp
import jax

IsTPU = False
batch_size = 64
dtype = jnp.float16
speech_path = '/home/zhenlan/Desktop/Projects/Bengali ASR/dlsprint/validation_files/'
data_path = '/home/zhenlan/Desktop/Projects/Bengali ASR/dlsprint/validation.csv'

model_path = '/home/zhenlan/Desktop/Projects/Bengali ASR/model_all3'
num_workers = 8
output_path = ''

In [2]:
#from whisper_jax import FlaxWhisperForConditionalGeneration
from transformers import FlaxWhisperForConditionalGeneration
from functions import *
from functools import partial
import optax
import evaluate
from jax import random
from transformers import AutoTokenizer
import time
import warnings
warnings.filterwarnings('ignore')

In [3]:
pad_to_multiple_of = 1
max_length_gen = 24
epochs = 1
verbose = 5
learning_rate=4e-4
clip = 1e-2
# tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v2", language="bn", task="transcribe")
#tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5")
tokenizer.bos_token = tokenizer.bos_token_id = None
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v2")
text = pd.read_csv(data_path)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


In [4]:
dataset1 = AudioDataset(text,speech_path,lambda x:x.path,\
                       None,orig_sr=32000, target_sr=16000)
train_loader = DataLoader(dataset1, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn,tokenizer=tokenizer,feature_extractor=feature_extractor,\
                                           pad_to_multiple_of=pad_to_multiple_of,IsTrain=True,IsTPU=IsTPU,batch_size=batch_size))

dataset2 = AudioDataset(text,speech_path,lambda x:x.path,\
                       None,orig_sr=32000, target_sr=16000)
train_loader2 = DataLoader(dataset2, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn,tokenizer=tokenizer,feature_extractor=feature_extractor,\
                                           pad_to_multiple_of=pad_to_multiple_of,IsTrain=False,IsTPU=IsTPU,batch_size=batch_size))
# dataset = AudioDataset(text.iloc[960000:],speech_path)
# test_loader = DataLoader(dataset, batch_size=batch_size*4, shuffle=False, num_workers=num_workers, \
#                         collate_fn=partial(collate_fn,tokenizer=tokenizer,feature_extractor=feature_extractor,\
#                                            pad_to_multiple_of=pad_to_multiple_of,IsTrain=True,IsTPU=IsTPU,batch_size=batch_size))

In [5]:
# load the processor and model
model, params = FlaxWhisperForConditionalGeneration.from_pretrained(
    model_path, dtype=dtype, _do_init=False)
# params = model.params

model.config.forced_decoder_ids = None
model.config.bos_token_id = None
model.config.suppress_tokens = None
model.config.decoder_start_token_id = None
model.generation_config.decoder_start_token_id = [50258, 50302, 50359, 50363]# '<|startoftranscript|><|bn|><|transcribe|><|notimestamps|>
model.generation_config.forced_decoder_ids = None
"""A list of pairs of integers which indicates a mapping from generation indices to token indices 
that will be forced before sampling. For example, [[0, 123]] means the first generated token 
will always be a token of index 123."""
model.generation_config.suppress_tokens = None
model.generation_config.begin_suppress_tokens = None
model.generation_config.bos_token_id = None

In [6]:
# @partial(jax.pmap,axis_name='data',in_axes=(None,None,0,0,0,None),out_axes=(None,None,None))
# def train_one_step_embed(embedding,params,audio,input_ids,attention_mask,opt_states):
#     def loss_fn(embedding,params,audio,input_ids,attention_mask):
#         params['model']['decoder']['embed_tokens']['embedding'] = embedding
#         out = model(audio,input_ids,decoder_attention_masjn_mask):
#         params['model']['decoder']['embed_tokens']['embedding'] = embedding
#         out = model(audio,input_ids,decoder_attention_mask=attention_mask,params=params,train=True).logits # (B, L, d)
#         return jnp.mean(optax.softmax_cross_entropy_with_integer_labels(out[:,3:-1], input_ids[:,4:])*attention_mask[:,4:])
#     grad_fn = jax.value_and_grad(loss_fn,has_aux=False)
#     l,grads = grad_fn(embedding,params,audio,input_ids,attention_mask)
#     updates, opt_states = opt.update(grads, opt_states,params=embedding)
#     embedding = optax.apply_updates(embedding, updates)
#     return embedding,opt_states,l

# @jax.jit
def eval_one_step(params,audio,input_ids,attention_mask):
    out = model(audio,input_ids,decoder_attention_mask=attention_mask,params=params,train=False).logits # (B, L, d)
    return jnp.mean(optax.softmax_cross_entropy_with_integer_labels(out[:,3:-1], input_ids[:,4:])*attention_mask[:,4:])

#@jax.jit
# def train_one_step(params,audio,input_ids,attention_mask,opt_states):
#     def loss_fn(params,audio,input_ids,attention_mask):
#         out = model(audio,input_ids,decoder_attention_mask=attention_mask,params=params,train=True).logits # (B, L, d)
#         return jnp.mean(optax.softmax_cross_entropy_with_integer_labels(out[:,3:-1], input_ids[:,4:])*attention_mask[:,4:])
#     grad_fn = jax.value_and_grad(loss_fn,has_aux=False)
#     l,grads = grad_fn(params,audio,input_ids,attention_mask)
#     updates, opt_states = opt.update(grads, opt_states,params=params)
#     params = optax.apply_updates(params, updates)
#     return params,opt_states,l

# @jax.jit
def generate(audio):
    return model.generate(audio,params=params,max_length=max_length_gen, num_beams=1, do_sample=False).sequences

metric = evaluate.load("wer")
def metric_one_step(audio,txt):
    generated_ids = generate(audio)
    transcriptions = tokenizer.batch_decode(generated_ids.tolist(), skip_special_tokens=True)
    wer = metric.compute(predictions=transcriptions, references=txt)
    return wer

# def batch_generate(loader):
#     pass
#     #transcriptions = [txt + "|" for txt in transcriptions]

In [7]:
%%time
# # model_all3
test_loss = 0
for j,(audio,input_ids,attention_mask) in enumerate(train_loader):
    #audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
    l = eval_one_step(params,audio,input_ids,attention_mask)
    test_loss += l.item()
    if j>20:
        break
test_loss /= (j+1)
print(test_loss)

0.4285888671875
CPU times: user 48min 35s, sys: 22min 36s, total: 1h 11min 11s
Wall time: 7min 28s


In [8]:
%%time
# model_all3
test_loss = 0
for j,(audio,txt) in enumerate(train_loader2):
    #audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
    l = metric_one_step(audio,txt)
    test_loss += l
    if j>20:
        break
test_loss /= (j+1)
print(test_loss)

0.3622065277408587
CPU times: user 1h 8min 18s, sys: 24min 38s, total: 1h 32min 57s
Wall time: 12min 31s
